## Taller de agentes con LangGraph

En este taller práctico, crearemos un agente usando la librería LangGraph y modelos de OpenAI. Este agente tendrá dos herramientas a su disposición:

*   **Tavily**: para búsquedas web
*   **WeatherAPI**: para previsión del tiempo (o tiempo actual)

El agente detectará la intencionalidad de la pregunta, decidirá si debe usar alguna de estas herramientas, y devolverá una respuesta al usuario construida a partir de la información recuperada de estas APIs

### Instalamos las librerías necesarias: paquetes específicos de langchain

In [ ]:
!pip install -q langgraph langchain langchain-community langchain-openai python-dotenv

### Importamos las herramientas necesarias, que todas están integradas en langchain

In [ ]:
import os
import requests
from typing import List, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI

### Definimos las APIs necesarias, tanto de OpenAI como de WeatherAPI

Recordemos que en una aplicación real, las APIs no deben quedar expuestas en el código

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

### Empezamos definiendo las tools (herramientas) que va a tener a su disposición el agente

Se usa el decorador @tool para indicarle a langchain que son herramientas. Al incluir este decorador, se extraen metadatos importantes como el nombre de la herramienta (que por defecto es el nombre de la función) y la descripción (el docstring de la función, la primera línea que se define entre comillas)


In [ ]:
@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current or forecast weather""" #docstring para darle la descripción
    endpoint = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={query}"
    response = requests.get(endpoint)
    data = response.json()

    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

@tool
def search_web(query: str) -> list:
    """Search the web for a query""" #docstring para darle la descripción
    tavily_search = TavilySearchResults(max_results=5, search_depth='advanced', max_tokens=1000)
    results = tavily_search.invoke(query)
    return results

### Probamos manualmente la API de get_weather

In [ ]:
get_weather("Madrid")

### Probamos manualmente la API de Tavily

In [ ]:
search_web("¿cuando inicia la temporada de la NFL 2026?")

### Llamamos al modelo con el parámetro bind_tools(), que permite pasar tools de Langchain para que las use el LLM

In [ ]:
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_ENDPOINT

llm = AzureChatOpenAI(azure_deployment="gpt-4o",
    api_key = OPENAI_API_KEY,
    api_version=API_VERSION,
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

tools = [search_web, get_weather]
llm_with_tools = llm.bind_tools(tools)

### El LLM por si mismo no tiene acceso a información en tiempo real

In [ ]:
query = "What is the current weather in Segovia today?"
response = llm.invoke(query)
print(response.content)

In [ ]:
query = "quien fue el campeon del super bowl en 2025?"
response = llm.invoke(query)
print(response.content)

### Podemos definir el prompt que se le pasará al LLM+tools, explicándole la tarea o tareas que queremos que haga

In [ ]:
prompt = """
    Given the tools at your disposal, mention tool calls for the following tasks --If any answer can´t be provided from tool calls, use your own knowledge:
    Do not change the query given for any search tasks
        1. What is the current weather in Madrid today
        2. Can you tell me about Kerala
        3. Why is the sky blue?
    """

results = llm_with_tools.invoke(prompt)

print(results.tool_calls)

In [ ]:
print(results)

In [ ]:
results.content

### Ahora creamos el agente con grafos, usando LangGraph

En primer lugar, usamos el módulo create_react_agent. El paradigma ReAct está bastado en Reason & Act, es decir, razonar (¿qué tool tengo a mi disposición y cuál debería usar?) y actuar (voy a usar esta herramienta para que me devuelva una respuesta y analizarla)

In [ ]:
from langgraph.prebuilt import create_react_agent

Creamos un system_prompt, donde le explicamos qué esperamos que haga, y las tools que tiene a su disposición. Es importante que los nombres de las tools coincidan con el nombre con el que le hemos definido, así como darle una buena explicación de su función

In [ ]:
system_prompt = """Act as a helpful assistant.
    Use the tools at your disposal to perform tasks as needed.
        - get_weather: whenever user asks get the weather of a place.
        - search_web: whenever user asks for information on current events or if you don't know the answer.
    Use the tools only if you don't know the answer."""

### Inicializamos el agente con el modelo, las tools y el system_prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt), ("placeholder", "{messages}")])

agent = create_react_agent(model=llm, tools=tools, prompt=prompt_template)

### Probamos el agente para ver qué hace

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "What is the current weather in Madrid today")]}

print_stream(agent.stream(inputs, stream_mode="values"))

### Vamos a crear la lógica de grafos (estados y nodos)

En primer lugar, definimos el conjunto de tools posibles mediante el ToolNode

In [ ]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

tools = [search_web, get_weather]
tool_node = ToolNode(tools)

Definimos las funciones para llamar al LLM y las tools. La función call_tools llevará al final (END)

In [ ]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tools(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

Inicializamos el flow con StateGraph

In [ ]:
workflow = StateGraph(MessagesState)

Ahora ya podemos añadir nodos. Primero, añadimos un nodo de LLM, que usará el LLM para tomar decisiones en base al input

In [ ]:
workflow.add_node("LLM", call_model)

Nuestro flow comienza con el nodo LLM. Esto se codifica añadiendolo con la función add_edge, indicando que es START

In [ ]:
workflow.add_edge(START, "LLM")

Después, pasamos por el nodo de las tools

In [ ]:
workflow.add_node("tools", tool_node)

Añadimos un edge condicional, que va del LLM a las llamadas de las tools. Dependiendo del output del LLM, irá al nodo de tools o al final (END), según el agente considere que necesita consultar una de las tools o puede dar una respuesta autónoma

In [ ]:
workflow.add_conditional_edges("LLM", call_tools)

Por último, el nodo tools le manda la información de vuelta al LLM

In [ ]:
workflow.add_edge("tools", "LLM")

Cuando hemos definido todo nuestro flow, se debe compilar

In [ ]:
agent = workflow.compile()

Se puede pintar el flow automáticamente

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph().draw_mermaid_png()))

### Ya podemos usar nuestro agente y ver los pasos intermedios que sigue

In [ ]:
for chunk in agent.stream(
    {"messages": [("user", "What was the stock price of BBVA in the Spanish market as of March 6th, 2025? And will I need an umbrella in Madrid for tomorrow? y quien fue el campeon del super bowl 2025?")]}, #"Will I need an umbrella in Madrid tomorrow?"
    stream_mode="values",):
    chunk["messages"][-1].pretty_print()